In [13]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download -d abdallahwagih/spam-emails

mkdir: cannot create directory ‘/root/.kaggle’: File exists
  0% 0.00/207k [00:00<?, ?B/s]
100% 207k/207k [00:00<00:00, 118MB/s]


In [16]:
from zipfile import ZipFile
with ZipFile("/content/spam-emails.zip","r") as zip:
  zip.extractall("/content/")

In [5]:
import pandas as pd
import numpy as np

In [17]:
email = pd.read_csv("spam.csv",encoding = "latin")

In [18]:
email.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [19]:
email.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Category  5572 non-null   object
 1   Message   5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [21]:
email.describe()

,Category,Message
count,5572,5572
unique,2,5157
top,ham,"Sorry, I'll call later"
freq,4825,30


In [27]:
email["spam"] = email["Category"].apply(lambda x : 1 if x=="spam" else 0)

In [28]:
email.head()

,Category,Message,spam
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


In [29]:
email.spam.value_counts()

0    4825
1     747
Name: spam, dtype: int64

In [39]:
from sklearn.model_selection import train_test_split

xtrain,xtest, ytrain,ytest = train_test_split(email['Message'], email['spam'], test_size=.25)

In [30]:
#Bag of word model
from sklearn.feature_extraction.text import CountVectorizer
counts = CountVectorizer()

In [41]:
xtrain_vector = counts.fit_transform(xtrain).toarray()

In [42]:
xtrain_vector[:1]

array([[0, 0, 0, ..., 0, 0, 0]])

In [43]:
from sklearn.linear_model import LogisticRegression

In [60]:
lr = LogisticRegression(class_weight="balanced")
lr.fit(xtrain_vector,ytrain)

LogisticRegression(class_weight='balanced')

In [47]:
ypred = lr.predict(counts.transform(xtest))

In [61]:
from sklearn.metrics import classification_report
print(classification_report(ytest, ypred))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      1209
           1       0.96      0.90      0.93       184

    accuracy                           0.98      1393
   macro avg       0.97      0.95      0.96      1393
weighted avg       0.98      0.98      0.98      1393



In [53]:
from sklearn.pipeline import Pipeline

lr_pipe = Pipeline([("Vectorizer",counts),("Logistic Regression",lr)])

In [55]:
lr_pipe.fit(xtrain,ytrain)

Pipeline(steps=[('Vectorizer', CountVectorizer()),
                ('Logistic Regression', LogisticRegression())])

In [56]:
ypred = lr_pipe.predict(xtest)

In [57]:
print(classification_report(ytest, ypred))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      1209
           1       0.96      0.90      0.93       184

    accuracy                           0.98      1393
   macro avg       0.97      0.95      0.96      1393
weighted avg       0.98      0.98      0.98      1393

